# Сессия 4

---

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

Чтение данных из csv файла

In [2]:
df = pd.read_csv("ndata.csv", encoding='utf-8-sig')

In [3]:
df

,district_name,sidebar.id,payload.dt,sidebar.author_name,sidebar.latitude,sidebar.longitude,sidebar.full_address,sidebar.responsible.data.executor.organization_name,sidebar.responsible.data.executor.person_position,sidebar.responsible.data.executor.person_name,...,reason.name,reason.category.city_object.name,payload.body_answer,reason.category.name,appeal,clean_appeal,clean_sentence,dayofweek,KMeans Clus Label,Clusters_name
0,Адмиралтейский,3100308,2020-12-28 11:36:41.194118,Павел Х.,59.928013,30.308225,Неизвестно,"СПб ГКУ ""Центр комплексного благоустройства""",Заместитель директора,Олехнович Виталий Павлович,...,"Лужи на проезжей части, тротуаре",Улица,Уважаемый пользователь!\n\nВ соответствии с Кл...,Благоустройство,лужа на пешеходном переходе,"['пешеходный', 'переход']",пешеходный переход,Monday,0,Назкаонные постройки
1,Кировский,3183624,2021-02-10 08:59:17.322348,Максим К.,59.860733,30.236146,"г.Санкт-Петербург, проспект Маршала Жукова, до...",СПб ГКУ ЖА Кировского района,Заместитель директора,Грибанова Мария Николаевна,...,Хранение вещей на лестничной площадке,Дом,Уважаемый пользователь!\r\n\r\nВ ответ на Ваше...,Содержание МКД,"Маршала Жукова 22, 14 этаж.\n\nБольше 2-х неде...","['маршал', 'жуков', 'этажбольше', 'неделя', 'х...",маршал жуков этажбольше неделя храниться мебел...,Wednesday,0,Назкаонные постройки
2,Кировский,2982887,2020-11-26 10:23:46.738623,Андрей Г.,59.849512,30.253931,Неизвестно,Неизвестно,Неизвестно,Неизвестно,...,Лужи на твердом покрытии,Двор,Уважаемый пользователь!\r\n\r\nСкопление осадк...,Благоустройство,На пешеходной дорожке от угла дома 21к2 в стор...,"['пешеходный', 'дорожка', 'сторона', 'детский'...",пешеходный дорожка сторона детский площадка по...,Thursday,0,Назкаонные постройки
3,Невский,3268824,2021-04-07 12:32:29.736374,Юлия Н.,59.888663,30.420071,"г.Санкт-Петербург, улица Седова, дом 35, литера А",Неизвестно,Неизвестно,Неизвестно,...,Неудовлетворительное состояние парадной,Дом,Уважаемый пользователь!\r\n\r\nРассмотрев Ваше...,Содержание МКД,Год назад был произведен ремонт парадной. Уже ...,"['назад', 'произвести', 'ремонт', 'парадный', ...",назад произвести ремонт парадный стена около к...,Wednesday,0,Назкаонные постройки
4,Курортный,3115412,2021-01-07 17:06:20.127364,Шшш Ш.,60.113166,29.969470,Неизвестно,"СПб ГКУ ""Центр комплексного благоустройства""",Заместитель директора,Нарджиева Наталья Николаевна,...,Неубранный снег,Мост,Уважаемый пользователь!\r\n\r\nНа указанной в ...,Благоустройство,Не убран снег,['убрать'],убрать,Thursday,3,Вандализм
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21215,Московский,3290309,2021-04-07 16:32:06.989784,Евгений К.,59.863719,30.326661,Неизвестно,Неизвестно,Неизвестно,Неизвестно,...,Неудовлетворительное состояние асфальтового по...,Двор,Уважаемый пользователь!\r\n\r\nАварийность асф...,Благоустройство,Ямы на асфальтном покрытии.,"['асфальтный', 'покрытие']",асфальтный покрытие,Wednesday,0,Назкаонные постройки
21216,Центральный,3057695,2020-12-09 12:47:45.812630,Владислав Б.,59.919059,30.347434,"г.Санкт-Петербург, Боровая улица, дом 26-28, л...",Неизвестно,Неизвестно,Неизвестно,...,Несанкционированные надписи / объявления на ст...,Дом,Уважаемый пользователь!\r\n\r\nРассмотрев Ваше...,Благоустройство,Прошу закрасить граффити.,"['просить', 'закрасить', 'граффити']",просить закрасить граффити,Wednesday,0,Назкаонные постройки
21217,Московский,3088031,2020-12-23 17:18:18.362936,Сергей Н.,59.866921,30.323337,Неизвестно,Неизвестно,Неизвестно,Неизвестно,...,"Неудовлетворительное состояние колодца, люка",Территория Санкт-Петербурга,Адрес: территория Московского парка Победы (ЗН...,Повреждения или неисправность элементов улично...,неудовлетворительное состояние покрытия ...,"['неудовлетворительный', 'состояние', 'покрыти...",неудовлетворительный состояние покрытие возле,Wednesday,0,Назкаонные постройки
21218,Фрунзенский,3030156,2020-12-10 09:53:29.340597,Юрий М.,59.829287,30.405238,Неизвестно,Администрация Фрунзенского ра

## 4.1. Разбиение набора данных на обучающую и тестовую выборки

Разобью данные 67% на 33% так как сичтаю это достаточным для обучения и тестирования

In [4]:
from sklearn.model_selection import train_test_split
#Использовал параметр stratify для того что бы какждый вид кластера присутсвтовал в обучающей и тестовой выборке
X_train, X_test, y_train, y_test = train_test_split(df[['district_name', 'sidebar.author_name','clean_sentence']], df['KMeans Clus Label'], test_size=0.33, random_state=42, stratify=df['KMeans Clus Label'])

## 4.2. Создание дополнительных признаков

Добавлю атрибут который будет содержать количество слов из очещеного предложения

In [5]:
#Использую метод len для подщета количетсва элементов в списке который получу от метода split
df["word_count"] = df['clean_sentence'].apply(lambda x: len(x.split(" ")))

Еще добавлю атрибут в котором будет находится средняя длинна слова в очищеном предложении

In [6]:
#Функция для подщета средней длины слова
def avg_word(sentence):
    words = sentence.split()
    return (sum(len(word) for word in words)/len(words))
#применяю функцию через apply
df['avg_word'] = df['clean_sentence'].apply(lambda x: avg_word(x))

## 4.3. Классификация обращений граждан по категориям обращений

Для обьедененя разнотипных атрибутов воспользуюсь модулем Pipeline

####  Добавление атрибутьов будет происходить при помощи модуля Pipeline

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn import metrics

# text and numeric classes that use sklearn base libaries
class TextTransformer(BaseEstimator, TransformerMixin):
    """
    Transform text features
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None, *parg, **kwarg):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberTransformer(BaseEstimator, TransformerMixin):
    """
    Transform numeric features
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]


# use the term-frequency inverse document frequency vectorizer to transfrom count of text
# into a weighed matrix of term importance
vec_tdidf = TfidfVectorizer(ngram_range=(1,1), analyzer='word', norm='l2')

# compile both the TextTransformer and TfidfVectorizer 
# to the text 'Text_Feature' 
text_sentence = Pipeline([
                ('transformer', TextTransformer(key='clean_sentence')),
                ('vectorizer', vec_tdidf)
                ])

# compile the NumberTransformer to 'text_numeric'
# and 'text_numeric2' numeric features
text_numeric = Pipeline([
                ('transformer', NumberTransformer(key='word_count')),
                ])

text_numeric2 = Pipeline([
                ('transformer', NumberTransformer(key='avg_word')),
                ])



# combine all of the features, text and numeric together
features = FeatureUnion([('text_attribute', text_sentence),
                      ('word_count', text_numeric),
                      ('avg_word', text_numeric2),
                      ])


text_numeric_features = ['clean_sentence', 'word_count', 'avg_word']
predictor = 'reason.category.name'

X_train, X_test, y_train, y_test = train_test_split(df[text_numeric_features], df[predictor], 
                                                    test_size=0.25, random_state=42, stratify=df[predictor])

#### для задачи классификации решил взять алгоритмы RandomForestClassifier, KNN и AdaBoost так как они хорошо подходят для обработки нескольких признаков и являются довольно быстрыми, что упрастит подбор гипер параметров

### RandomForestClassifier

Настройка гипер параметров

In [8]:
# Minimum number of samples per leaf
n_estimators = [99, 200]

# Minimum number of samples to split a node
max_depth = [25, 50]

hyperparameter_grid = {'n_estimators': n_estimators,
                       'max_depth': max_depth}

RandomizedSearchCV это инструмент, который позволяет сделать прогон циклом по нескольким гиперпараметрам и выбрать лучший

In [9]:
from sklearn.ensemble import RandomForestClassifier

# Create the model to use for hyperparameter tuning
model = RandomForestClassifier()

# Set up the random search with 4-fold cross validation
clf = RandomizedSearchCV(estimator=model,
                               param_distributions=hyperparameter_grid,
                               cv=3, n_iter=20, 
                               scoring = 'neg_mean_absolute_error',
                               n_jobs = -1, verbose = 1, 
                               return_train_score = True,
                               random_state=42)

# unite the features and classfier together
pipe = Pipeline([('features', features),
                 ('clf',clf)
                 ])

Обучение модели

In [10]:
# fit the model
pipe.fit(X_train, y_train)

# unite the features and classfier together
pipeforst2 = Pipeline([('features', features),
                 ('clf',clf.best_estimator_)
                 ])

# fit the model
pipeforst2.fit(X_train, y_train)

# predict from the test set
preds = pipeforst2.predict(X_test)

# print out your accuracy
from sklearn.metrics import classification_report
print("best was - ", clf.best_estimator_)
print(classification_report(y_test, preds))

Fitting 3 folds for each of 4 candidates, totalling 12 fits
best was -  RandomForestClassifier(max_depth=25, n_estimators=99)
                                                                                  precision    recall  f1-score   support

                                                                 Благоустройство       0.68      0.99      0.81      2853
                                                                   Водоотведение       1.00      0.03      0.06        30
                                                                   Водоснабжение       0.95      0.19      0.32        98
                                                                          Кровля       0.00      0.00      0.00        79
                                  Нарушение порядка пользования общим имуществом       0.00      0.00      0.00        22
                                   Нарушение правил пользования общим имуществом       1.00      0.37      0.54       199
                   

метрики - weighted accuracy macro по F1-score показывают не плохой результат.

### KNN

Настройка гипер параметров

In [11]:
# Minimum number of samples per leaf
n_neighbors = [4, 7]

# Minimum number of samples to split a node
leaf_size = [29, 69]

hyperparameter_grid = {'n_neighbors': n_neighbors,
                       'leaf_size': leaf_size}

Так же использую RandomizedSearchCV для подбора нескольких параметров

In [12]:
from sklearn.neighbors import KNeighborsClassifier
# Create the model to use for hyperparameter tuning
model = KNeighborsClassifier()

# Set up the random search with 4-fold cross validation
clf = RandomizedSearchCV(estimator=model,
                               param_distributions=hyperparameter_grid,
                               cv=3, n_iter=20, 
                               scoring = 'neg_mean_absolute_error',
                               n_jobs = -1, verbose = 1, 
                               return_train_score = True,
                               random_state=42)

# unite the features and classfier together
pipe = Pipeline([('features', features),
                 ('clf',clf)
                 ])

Обучение модели

In [13]:
# fit the model
pipe.fit(X_train, y_train)

# unite the features and classfier together
pipe2 = Pipeline([('features', features),
                 ('clf',clf.best_estimator_)
                 ])

# fit the model
pipe2.fit(X_train, y_train)

# predict from the test set
preds = pipe2.predict(X_test)

# print out your accuracy
print("best was - ", clf.best_estimator_)
print(classification_report(y_test, preds))

Fitting 3 folds for each of 4 candidates, totalling 12 fits
best was -  KNeighborsClassifier(leaf_size=29, n_neighbors=4)
                                                                                  precision    recall  f1-score   support

                                                                 Благоустройство       0.78      0.88      0.83      2853
                                                                   Водоотведение       0.32      0.20      0.24        30
                                                                   Водоснабжение       0.58      0.27      0.36        98
                                                                          Кровля       0.67      0.30      0.42        79
                                  Нарушение порядка пользования общим имуществом       0.50      0.05      0.08        22
                                   Нарушение правил пользования общим имуществом       0.89      0.74      0.81       199
                       

На удивление KNN справился хуже с этой задачей

### AdaBoost

Настройка гипер параметров

In [14]:
# Minimum number of samples per leaf
n_estimators = [49, 100]

# Minimum number of samples to split a node
learning_rate = [1.1, 3]

hyperparameter_grid = {'n_estimators': n_estimators,
                       'learning_rate': learning_rate}

Так же использую RandomizedSearchCV для подбора нескольких параметров

In [15]:
from sklearn.ensemble import AdaBoostClassifier
# Create the model to use for hyperparameter tuning
model = AdaBoostClassifier()

# Set up the random search with 4-fold cross validation
clf = RandomizedSearchCV(estimator=model,
                               param_distributions=hyperparameter_grid,
                               cv=3, n_iter=20, 
                               scoring = 'neg_mean_absolute_error',
                               n_jobs = -1, verbose = 1, 
                               return_train_score = True,
                               random_state=42)

# unite the features and classfier together
pipe = Pipeline([('features', features),
                 ('clf',clf)
                 ])

Обучение модели

In [16]:
# fit the model
pipe.fit(X_train, y_train)

# unite the features and classfier together
pipe2 = Pipeline([('features', features),
                 ('clf',clf.best_estimator_)
                 ])

# fit the model
pipe2.fit(X_train, y_train)

# predict from the test set
preds = pipe2.predict(X_test)

# print out your accuracy
print("best was - ", clf.best_estimator_)
print(classification_report(y_test, preds))

Fitting 3 folds for each of 4 candidates, totalling 12 fits
best was -  AdaBoostClassifier(learning_rate=1.1, n_estimators=49)
                                                                                  precision    recall  f1-score   support

                                                                 Благоустройство       0.68      0.90      0.77      2853
                                                                   Водоотведение       0.62      0.43      0.51        30
                                                                   Водоснабжение       0.92      0.59      0.72        98
                                                                          Кровля       0.00      0.00      0.00        79
                                  Нарушение порядка пользования общим имуществом       0.09      0.59      0.16        22
                                   Нарушение правил пользования общим имуществом       0.51      0.46      0.48       199
                  

AdaBoost показал самый плохой результат по итогам метрик

#### Лучшим из этих алгормитмов классификаци для данной задачи является  RandomForestClassifier

### 4.4. Подготовка отчета

In [23]:
df

,district_name,sidebar.id,payload.dt,sidebar.author_name,sidebar.latitude,sidebar.longitude,sidebar.full_address,sidebar.responsible.data.executor.organization_name,sidebar.responsible.data.executor.person_position,sidebar.responsible.data.executor.person_name,...,payload.body_answer,reason.category.name,appeal,clean_appeal,clean_sentence,dayofweek,KMeans Clus Label,Clusters_name,word_count,avg_word
0,Московский,3039943,2020-12-07 11:37:10.685301,Сергей А.,59.844490,30.323054,Неизвестно,Комитет по печати и взаимодействию со средства...,Заместитель председателя Комитета,Крутоверцев Артем Георгиевич,...,"По данному адресу нарушения устранены, рекламн...",Незаконная информационная и (или) рекламная ко...,Незаконная реклама на стендах информации,"['незаконный', 'реклама', 'стенд', 'информация']",незаконный реклама стенд информация,Monday,0,Назкаонные постройки,4,8.000000
1,Невский,3317136,2021-04-09 11:16:01.604339,Артем Я.,59.911977,30.473145,"г.Санкт-Петербург, улица Подвойского, дом 17, ...",СПб ГКУ ЖА Невского района,Директор,Середин Владимир Ипполитович,...,"В результате проверки, проведённой Исполнителе...",Санитарное состояние,"Сильный неприятный запах из подвала дома, под...","['сильный', 'неприятный', 'запах', 'подвал', '...",сильный неприятный запах подвал подвал сырой в...,Friday,0,Назкаонные постройки,20,7.450000
2,Василеостровский,3010372,2020-12-02 10:09:58.362773,Алекса Р.,59.944655,30.272977,"г.Санкт-Петербург, 9-я линия В.О., дом 52, лит...",Администрация Василеостровского района,Заместитель главы администрации,Родионов Владлен Викторович,...,Уважаемый пользователь!\nРассмотрев Ваше сообщ...,Нарушение правил пользования общим имуществом,незаконный воздуховод в оконном проёме.,"['незаконный', 'воздуховод', 'оконный']",незаконный воздуховод оконный,Wednesday,0,Назкаонные постройки,3,9.000000
3,Адмиралтейский,3273250,2021-03-30 12:25:32.292350,Виктория Д.,59.920305,30.287807,"г.Санкт-Петербург, набережная канала Грибоедов...",СПб ГКУ ЖА Адмиралтейского района,Директор,Кузнецов Андрей Юрьевич,...,Уважаемый пользователь!\n\nРассмотрев Ваше обр...,Фасад,сняли табличку с адресом и номером дома по Гри...,"['снять', 'табличка', 'адрес', 'номер', 'грибо...",снять табличка адрес номер грибоедов,Tuesday,0,Назкаонные постройки,5,6.400000
4,Пушкинский,3322619,2021-04-22 14:37:38.298759,Юрий С.,59.716047,30.411032,"город Пушкин, Малая улица, дом 64, литера А",СПб ГКУ ЖА Пушкинского района,Заместитель директора,Кошмак Ольга Петровна,...,Уважаемый Пользователь!\r\n\r\n Рассмотрев Ваш...,Фасад,Необходимо отчистить стену при входе в 5 парад...,"['необходимо', 'отчистить', 'стена', 'парадный']",необходимо отчистить стена парадный,Thursday,0,Назкаонные постройки,4,8.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5362,Красногвардейский,3185003,2021-02-08 12:56:02.219594,Александр С.,59.940807,30.488387,"г.Санкт-Петербург, проспект Наставников, дом 1...",СПб ГКУ ЖА Красногвардейского района,Заместитель директора,Долгова Наталия Вячеславовна,...,Уважаемый пользователь!\n В связи с обращени...,Содержание МКД,"Необходимо убрать рекламу. 1 парадная, 7 этаж","['необходимо', 'убрать', 'реклама', 'парадный']",необходимо убрать реклама парадный,Monday,0,Назкаонные постройки,4,7.750000
5363,Красносельский,3283376,2021-04-13 09:46:51.655711,Вероника А.,59.847737,30.105815,"г.Санкт-Петербург, Петергофское шоссе, дом 86,...",СПб ГКУ ЖА Красносельского района,Директор,Васильева Валентина Григорьевна,...,Уважаемый пользователь!\r\nНа момент проведени...,Содержание МКД,"В очередной раз фотографирую ОДНО И ТОЖЕ, соот...","['очередной', 'фотографировать', 'соотвественн...",очередной фотографировать соотвественный уборк...,Tuesday,0,Назкаонные постройки,5,10.400000
5364,Центральный,3285431,2021-03-29 10:54:22.068300,Мария В.,59.931510,30.353373,"г.Санкт-Петербург, Невский проспект, дом 65, л...",СПб ГКУ ЖА Центрального района,Директор СПб ГКУ ЖА Центрального района,Костыленко Владимир Григорьевич,...,Уважае

#### Было выполнено:
* 4.1. Разбиение набора данных на обучающую и тестовую выборки - Выборка была разбита на тестовую и обучающую со сбалансированым разделением и стратификацией по виду кластеров;
* 4.2. Создание дополнительных признаков - Было сформировано 2 атрибута (word_count и avg_word);
* 4.3. Классификация обращений граждан по категориям обращений - Была выполнена классификация несколькими способами (RandomForestClassifier, KNN и AdaBoost); Была выполнена нстройка гипер параметров при помощи RandomizedSearchCV; Оценка качества осуществлялась по средствам метрик weighted accuracy macro по F1-score; По результатам метрик был выбран лучший алгоритм (RandomForestClassifier);

In [29]:
df.to_csv("nddata.csv", index=False, encoding="utf-8-sig")

In [24]:
import pickle
pickle.dump(model, open("pickle.dat", "wb"))

---

In [ ]:
df2 = df[df["reason.category.name"] == "Благоустройство"][::14]
df3 = df[df["reason.category.name"] == "Содержание МКД"][::12]
df = df[df["reason.category.name"] != "Благоустройство"]
df = df[df["reason.category.name"] != "Содержание МКД"]
df = pd.concat([df, df2, df3])
df.reset_index(drop=True, inplace=True)
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
X3 = tfidf.fit_transform(df['clean_sentence'])
X_train, X_test, y_train, y_test = train_test_split(X3, df["reason.category.name"], test_size=0.33, random_state=42, stratify=df["reason.category.name"])
model = RandomForestClassifier()
model.fit(X_train, y_train)
preds = model.predict(X_test)
x = tfidf.transform(["необходимо отчистить стена парадный"])
model.predict(x)
import pickle
pickle.dump(model, open("pickle.dat", "wb"))